In [1]:
import math

import numpy as np

from keras import backend as K
from keras.layers import Convolution2D, Input
from keras.models import Model

Using TensorFlow backend.


In [2]:
class TriangularConvolution2D(Convolution2D):
    
    def __init__(self, *args, **kwargs):
        # TODO input filter size should reflet the triangle length L
        # then we can initialize the base type using 2*L + 1 
        super(TriangularConvolution2D, self).__init__(*args, **kwargs)

        self.mask = None
        
    def build(self, input_shape):
        super(TriangularConvolution2D, self).build(input_shape)

        # Create a numpy array of ones in the shape of our convolution weights.
        self.mask = np.ones(self.weights[0].shape)

        # We assert the height and width of our convolution to be equal as they should.
        assert self.mask.shape[0] == self.mask.shape[1]

        # for now let's make sure the filter width is odd number
        assert self.mask.shape[0] % 2 == 1
        
        # Since the height and width are equal, we can use either to represent the size of our convolution.
        filter_size = self.mask.shape[0]
        filter_center = filter_size // 2
        print(filter_center)

        # Zero out all weights above the center.
        self.mask[:filter_center, :, :, :] = 0

        # Zero out all weights to the right of the center.
        self.mask[:, (filter_center+1):, :, :] = 0
        
        # zero out the little triangle to the left bottom
        # TODO right now this is being done in a stupid way
        for i in range(filter_center, self.mask.shape[0]):
            self.mask[i, :(i-filter_center), :, :] = 0

        # zero out the center weigths
        self.mask[filter_center, filter_center, :, :] = 0

        # Convert the numpy mask into a tensor mask.
        self.mask = K.variable(self.mask)

    def call(self, x, mask=None):
        ''' I just copied the Keras Convolution2D call function so don't worry about all this code.
            The only important piece is: self.W * self.mask.
            Which multiplies the mask with the weights before calculating convolutions. '''
        output = K.conv2d(x, self.weights[0] * self.mask,     strides=(1, 1),
                        padding=self.padding, data_format=self.data_format,
                        dilation_rate=self.dilation_rate)
        
        if self.use_bias:
            output = K.bias_add(
                output,
                self.bias,
                data_format=self.data_format)
            
        if self.activation is not None:
            return self.activation(output)
        
#         if self.bias:
#             if self.dim_ordering == 'th':
#                 output += K.reshape(self.b, (1, self.nb_filter, 1, 1))
#             elif self.dim_ordering == 'tf':
#                 output += K.reshape(self.b, (1, 1, 1, self.nb_filter))
#             else:
#                 raise ValueError('Invalid dim_ordering:', self.dim_ordering)
#         output = self.activation(output)
        return output

    def get_config(self):
        # Add the mask type property to the config.
        return dict(list(super().get_config().items()) + list({'mask': self.mask_type}.items()))

In [3]:
shape = (50, 50, 1)
filters = 20

input_img = Input(shape)

output = TriangularConvolution2D(filters, 5, 5, border_mode='same')(input_img)


model = Model(input=input_img, output=output)

2


/Users/alicegao/anaconda2/envs/dg_work/lib/python2.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `TriangularConvolution2D` call to the Keras 2 API: `TriangularConvolution2D(20, (5, 5), padding="same")`
  
/Users/alicegao/anaconda2/envs/dg_work/lib/python2.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("tr..., inputs=Tensor("in...)`
  if __name__ == '__main__':


In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50, 50, 1)         0         
_________________________________________________________________
triangular_convolution2d_1 ( (None, 50, 50, 20)        520       
Total params: 520
Trainable params: 520
Non-trainable params: 0
_________________________________________________________________
